In [28]:
import tweepy
from tkinter import *
import tkinter
from time import sleep
from datetime import datetime
from textblob import TextBlob
import matplotlib.pyplot as plt

In [29]:
#Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
def plotfunc(tweet,replies):
    polarity_list = []
    numbers_list = []
    number = 0
    for i in replies:
        try:
            print('analysing for sentiment analysis')
            analysis = TextBlob(i.full_text)
            analysis = analysis.sentiment
            polarity = analysis.polarity
            polarity_list.append(polarity)
            numbers_list.append(number)
            number = number + 1
        except tweepy.TweepError as e:
            print(e.reason)
        except StopIteration:
            break
    
#Plotting
    axes = plt.gca()
    axes.set_ylim([-1, 2])

    plt.scatter(numbers_list, polarity_list)

    averagePolarity = (sum(polarity_list))/(len(polarity_list))
    averagePolarity = "{0:.0f}%".format(averagePolarity * 100)
    time  = datetime.now().strftime("At: %H:%M\nOn: %m-%d-%y")

    plt.text(0, 1.25, "Average Sentiment:  " + str(averagePolarity) + "\n" + time, fontsize=12, bbox = dict(facecolor='none', edgecolor='black', boxstyle='square, pad = 1'))

    plt.title("Sentiment of"+tweet.text+"on Twitter")
    plt.xlabel("Number of Tweets")
    plt.ylabel("Sentiment")
    plt.show()

In [ ]:
import csv
replies=[] 
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

for full_tweets in tweepy.Cursor(api.user_timeline,screen_name='RahulGandhi',timeout=999999).items(50):
    for tweet in tweepy.Cursor(api.search,q='to:RahulGandhi', since_id=full_tweets.id, result_type='recent',timeout=999999).items(2000):
        if hasattr(tweet, 'in_reply_to_status_id_str'):
            if (tweet.in_reply_to_status_id_str==full_tweets.id_str):
                t = [tweet.text,tweet.author.screen_name,tweet.created_at]
                with open('comments5.csv','a') as f:
                    writer= csv.writer(f)
                    row = [str(full_tweets.id),tweet.text]
                    writer.writerows([row])
                    replies.append(tweet.text)
    print("Tweet :",full_tweets.text.translate(non_bmp_map))
    for elements in replies:
        print("Replies :",elements)
    replies.clear()

Tweet : NoMo Jobs! 

The Fuhrer promised us 2 Cr jobs a year. 5 years later,  his leaked job creation report card reveals a… https://t.co/d6mchyxTEh
Replies : @RahulGandhi It is an insult of the country's Prime Minister to call him Fuhrer. Mr. Gandhi Please learn some manners.
Replies : @RahulGandhi @ShuklaRajiv अरे भई क्या वास्तव में पप्पू हो?ये जो मुद्रा योजना के तहत लोग स्वावलंबी बने रोजगार नहीं तो क्या कहलायेगा
Replies : @RahulGandhi For those who ask what Congress did 40+ years, they were pivotal for today's modern India by starting… https://t.co/AOL2RBiQaG
Replies : @RahulGandhi Today, at the #BudgetSession in #Parliament 

#YONOSBI20U20  #Budget2019 #FridayFeeling #KalpanaChawla… https://t.co/Yf4Apzj4TG
Replies : @RahulGandhi रोजगार विरोधी गवर्नमेंट डबलबेंच के आदेश के खिलाफ कोर्ट पहुंची जिससे रोजगार छीना जा सके आज उसकी डेट लगी… https://t.co/Ewfi8QBX3p
Replies : @RahulGandhi Why are you shouting now, this government is in ruleing power for last 5year but your family had 70 ye… ht

Tweet : I totally empathise with Parrikar Ji's situation &amp; wish him well. He's under immense pressure from the PM after our… https://t.co/QtdSLOFzSR
Replies : @RahulGandhi @rssurjewala किसी अस्वस्थ व्यक्ति के साथ इस तरह की घटिया राजनीती से आपने अपने आप को जनता की नजरो में ए… https://t.co/TnV0jwDw3R
Replies : @RahulGandhi Pappu ghatiya insaan
Replies : @RahulGandhi Junior doing another Macron,making up  another conversation.
Replies : @RahulGandhi . https://t.co/Hvei2CMpRq
Replies : @RahulGandhi Some interesting fact is if we
Replies : @RahulGandhi https://t.co/jLtVF5a1Vy
Replies : @RahulGandhi How am I reading this letter without it being leaked?
Replies : @RahulGandhi We seen what matters more to your Gandhi family
Replies : @RahulGandhi Liar you.
Replies : @RahulGandhi You are a sick for sure. Being druggiest you can't write such a letter.
Replies : @RahulGandhi Just when you think a human being cannot be more pathetic....
Replies : @RahulGandhi Mr.Rahul ji,make ur own identity, 

Tweet : "एक समाज की महानता और प्रगति इस बात से लगायी जा सकती है कि वहां कमजोर और असुरक्षित सदस्यों के साथ कैसा व्यवहार किया… https://t.co/KxW7TKfDpP
Replies : @RahulGandhi Bapu to yad aya h siyasat ke liye kabhi to bhagat Singh ji or Rajguru ji Shah boss ji ko bhi yad kar l… https://t.co/gxuCkhSLu6
Replies : @RahulGandhi @INCIndia Jaise aap log vaise hi aap ke dost nice sir https://t.co/irUxN0X1xH
Replies : @RahulGandhi @Ekta_79 राहुल जी हम आपकी विचारधारा जानना चाहते है?
Replies : @RahulGandhi @PriyaankaGandhi बिल्कुल सही है आदरणीय https://t.co/fLUAHcdDy7
Replies : @RahulGandhi इदर बी हस रहा है... ये किस टाइप का चूतिया है.
Replies : @RahulGandhi Jai gandhiraj
Replies : @RahulGandhi अच्छे लोगों को collect करेंगे
जनता आप को select करेंगे
बापू को सादर नमन 🇮🇳🙏
Replies : @RahulGandhi राहुलजी आप अण्णा हजारे को मिळणे आयेईये
Replies : @RahulGandhi Sath sath naman
Replies : @RahulGandhi Winter is come and gone but grief returns with the revolving year.
Replies : @RahulGandhi Purey list me sabse

Tweet : This man is an embarrassment to every Indian. He's unfit to be a Union Minister and deserves to be sacked.  

https://t.co/SbrvEdQBur
Replies : @RahulGandhi Wht r u fit to b?
Replies : @RahulGandhi Kisi dusare ki majak udane me tume bahut maza aata hau
Replies : @RahulGandhi Wah 're bjp walo tumara pm gappu hai
Replies : @RahulGandhi देश की राजनीति का आपने स्तर तो गिराया हि  है  इसके साथ साथ अपने झुठ बोलने की महारथ हासिल कर ली है
  ध… https://t.co/AnCnmLnpXZ
Tweet : 30 days since the Goa Audio Tapes on RAFALE were released. No FIR or enquiry ordered. No action against the Ministe… https://t.co/worJNh03MY
Replies : @RahulGandhi Tell the truth to the nation why thete were middlemen always in defence deals during UPA? Lack of know… https://t.co/vKJH5M06QI
Replies : @RahulGandhi U are useless!!
Replies : @RahulGandhi https://t.co/ZE0NeMUbOQ

Watch it carefully and reply this first
Thanks
Replies : @RahulGandhi Who cares about POGO tape?
Replies : @RahulGandhi Guru aap hi FIR kar do

In [27]:
import pandas as pd
from textblob  import TextBlob
df = pd.read_csv('comments5.csv')
df['ID'].unique()
df2 = pd.DataFrame([str(x) for x in df['ID']])
df2[0].unique()

array(['1.09089104436177e+18', '1.09064009090318e+18',
       '1.09047525395075e+18', '1.09014987365402e+18',
       '1.09013002937426e+18', '1.09012783347141e+18',
       '1.08985486817564e+18', '1.08979017004658e+18'], dtype=object)